In [18]:
!pip install selenium==3.11.0

#### Selenium & Driver(for browser) 준비

- Selenium : http://selenium-python.readthedocs.io/ 에서 공식 문서 확인 가능
- **GhostDriver (PhantomJS)** : 크롬 드라이버와 달리 실제 화면을 띄우지 않은 상태에서 백그라운드에서 크롤링이 가능
- ChromeDriver & GhostDriver 다운로드 후 작업 폴더에 위치시키기 

In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup 
import time
import pandas as pd

<br>
<br>

## 1. Selenium 둘러보기

In [28]:
df = pd.read_excel('result_190729_0552.xlsx', encoding='utf-8')
df.head()

,Title,Date,Article,URL,PressCompany
0,"""6살 한국 유튜버 90억원 건물 매입"" CNN '보람튜브' 소개",2019.07.27. 오후 5:18,【서울=뉴시스】CNN이 27일 한국의 인기 아동 유튜브 콘텐츠 '보람채널'의 흥행을...,https://news.naver.com/main/read.nhn?mode=LSD&...,뉴시스
1,"CNN도 주목한 '보람튜브'…""아동 유튜버, 과거에도 논란""",2019.07.27. 오후 4:03,보람튜브 갈무리 © 뉴스1(서울=뉴스1) 이원준 기자 = 유명 유튜브 채널 '보람튜...,https://news.naver.com/main/read.nhn?mode=LSD&...,뉴스1
2,"CNN, '보람튜브' 주목… ""아동유튜버 논란 우려""",2019.07.27. 오후 5:39,/사진=보람튜브 유튜브채널 캡처유명 유튜브채널 ‘보람튜브’에 대해 외신들도 주목하기...,https://news.naver.com/main/read.nhn?mode=LSD&...,머니S
3,'보람튜브 빌딩 매입'보다 주목해야 할 사실,2019.07.27. 오후 4:54,"키즈콘텐츠 성공, 영유아 시청시간에 비례…스마트폰 ‘과의존’ 아이들 늘었다 영유아기...",https://news.naver.com/main/read.nhn?mode=LSD&...,미디어오늘
4,[오태훈의 시사본부] 강남 빌딩 매입한 ‘보람튜브’ 고수익의 비밀,2019.07.26. 오후 3:42,- 보람튜브 위상 어마어마해…해외 크리에이터와 견줘도 뒤쳐지지 않는 글로벌 콘텐츠 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,KBS


In [29]:
driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe') 
# driver = webdriver.PhantomJS('(driver) phantomjs.exe')

translate_url = 'https://translate.google.co.kr/?um=1&ie=UTF-8&hl=ko&client=tw-ob#ko/en/' # 마지막 ko & en이 언어에 해당
driver.get(translate_url) 
print(driver.current_url)

https://translate.google.co.kr/?um=1&ie=UTF-8&hl=ko&client=tw-ob#ko/en/


<br>

## 문장 번역 & 크롤링

In [30]:
driver.find_element_by_id('source').send_keys('파이썬은 쉽습니다')

In [31]:
translated_contents = driver.find_element_by_class_name('tlid-translation')

print(translated_contents)
print(translated_contents.text)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"class name","selector":"tlid-translation"}
  (Session info: chrome=79.0.3945.79)
  (Driver info: chromedriver=2.37.544315 (730aa6a5fdba159ac9f4c1e8cbc59bf1b5ce12b7),platform=Windows NT 10.0.18362 x86_64)


<br>

## 기사글 번역 & 크롤링

In [ ]:
article = df['Article'][0]

driver.find_element_by_id('source').clear()
driver.find_element_by_id('source').send_keys(article)
time.sleep(3)

translated_contents = driver.find_element_by_class_name('tlid-translation').text
print('기사글 [ {} ] 의 번역이 끝났습니다.'.format(df['Title'][0]))
print(translated_contents)

In [ ]:
driver.close()
driver.quit()

<br>
<br>

## 2. 기사 원문 전체 번역 (시간이 오래 걸립니다 & 중간에 IP가 막힐 수 있습니다)

In [ ]:
# eng_contents = []

# driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe') 
# # driver = webdriver.PhantomJS('(driver) phantomjs.exe')

# translate_url = 'https://translate.google.co.kr/?um=1&ie=UTF-8&hl=ko&client=tw-ob#ko/en/' # 마지막 ko & en이 언어에 해당
# driver.get(translate_url) 
# print(driver.current_url)

# for row_index, row in df.iterrows():
#     driver.find_element_by_id('source').clear()
#     driver.find_element_by_id('source').send_keys(df['Article'][0])
#     time.sleep(3)

#     translated_contents = driver.find_element_by_class_name('tlid-translation').text
#     eng_contents.append(translated_contents)
#     print('기사글 [ {} ] 의 번역이 끝났습니다.'.format(df['Title'][row_index]))

# print('전체 contents 번역이 끝났습니다!')

# driver.close()
# driver.quit()


# df['Translated_article'] = eng_contents
# df.to_excel('translation_result.xlsx', index=False, encoding='utf-8')

# print('crawling_result.xlsx 파일로 전체 저장이 완료되었습니다!')

<br>
<br>

## 3. 한글 word_dict에서 상위 항목만 단어로 가져와 번역

In [26]:
from konlpy.tag import Twitter

articles = df['Article'].tolist()
articles = ''.join(articles)

twitter = Twitter()
raw_pos_tagged = twitter.pos(articles, norm=True, stem=True)

del_list = ['하다', '있다', '되다', '이다', '돼다', '않다', '그렇다', '아니다', '이렇다', '그렇다', '어떻다'] 
word_cleaned = []
for word in raw_pos_tagged:
    if not word[1] in ["Josa", "Eomi", "Punctuation", "Foreign"]: # Foreign == ”, “ 와 같이 제외되어야할 항목들
        if (len(word[0]) != 1) & (word[0] not in del_list): # 한 글자로 이뤄진 단어들을 제외 & 원치 않는 단어들을 제외
            word_cleaned.append(word[0])

word_counted = Counter(word_cleaned)
word_dic = dict(word_counted)
        
sorted_word_dic = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)
print(sorted_word_dic)

C:\Users\student\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


NameError: name 'Counter' is not defined

In [ ]:
# 빈도 순으로 상위 100개에 해당하는 단어까지만 추려내어 번역을 진행합니다. (늘릴 수 있습니다.)

translation_target = sorted_word_dic[:100] 
translation_target[-1] # 상위 100번째에 해당하는 단어의 빈도수를 확인합니다.

In [ ]:
translation_target = {}
for item in word_dic:
    if word_dic[item] >= 31: # 위에서 확인한 빈도수를 기반으로 번역 대상 단어를 선정합니다.
        translation_target[item] = word_dic[item]

In [ ]:
print(translation_target)

In [ ]:
translation_result = {}

driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe') 
# driver = webdriver.PhantomJS('(driver) phantomjs.exe')

translate_url = 'https://translate.google.co.kr/?um=1&ie=UTF-8&hl=ko&client=tw-ob#ko/en/' # 마지막 ko & en이 언어에 해당
driver.get(translate_url) 
print(driver.current_url)
    
for key in translation_target:
    driver.find_element_by_id('source').clear()
    driver.find_element_by_id('source').send_keys(key)
    time.sleep(1)

    translated_contents = driver.find_element_by_class_name('tlid-translation').text
    translation_result[translated_contents] = translation_target[key]
    
    print('단어 {}의 번역 완료 : {}'.format(key, translated_contents))

print('전체 번역이 끝났습니다!')

driver.close()
driver.quit()

In [25]:
translation_result['Bo-Ram'] = translation_result['Reward']
# translation_result['YouTuber'] = translation_result['Yu Tuber']
# translation_result['Lee Bo-Ram'] = translation_result['This']

del translation_result['Reward']
# del translation_result['Yu Tuber']
# del translation_result['This']

NameError: name 'translation_result' is not defined

In [24]:
from PIL import Image 
from wordcloud import WordCloud, ImageColorGenerator # Image 로부터 Color 를 생성(Generate)해내는 객체입니다.
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

youtube_coloring = np.array(Image.open("youtube_mask.jpg"))
image_colors = ImageColorGenerator(youtube_coloring)

word_cloud = WordCloud(font_path="C:/Windows/Fonts/malgun.ttf", # font_path="C:/Windows/Fonts/NanumSquareB.ttf"
                       width=2000, height=1000,  
                       mask=youtube_coloring, 
                       background_color='white').generate_from_frequencies(translation_result)

plt.figure(figsize=(15,15))
plt.imshow(word_cloud.recolor(color_func=image_colors), interpolation='bilinear') # 다시(re) 색칠하기
# plt.imshow(word_cloud.recolor(colormap='Blues'), interpolation='bilinear') # Matplotlib colormap 활용 (http://j.mp/32UXOQ6)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

NameError: name 'translation_result' is not defined